In [162]:
import pandas as pd

In [163]:
df = pd.read_csv('/mydata/wss-project-files/flight-delay-model/data/full.csv')
df = df.reset_index(drop=True)

In [164]:
def one_hot_encode(df: pd.DataFrame, column_name: str) -> pd.DataFrame:
    try:
        dummy1 = pd.get_dummies(df[column_name])
        dummy1 = dummy1.add_prefix(f"{column_name} - ")
        df = pd.concat([df, dummy1], axis=1).drop(column_name, axis=1)
        df.head()
    except Exception as e:
        print(f"Error in one_hot_encode: {e}")
    return df

In [165]:
one_hot_columns = [
    'Reporting_Airline', 'IATA_CODE_Reporting_Airline', 'DOT_ID_Reporting_Airline', 'Dest'
    ]
for col in one_hot_columns:
    df = one_hot_encode(df, col)

In [166]:
df = df.rename(columns={'Origin':'NAME', 'FlightDate':'DATE'})

In [167]:
df['DATE'] = pd.to_datetime(df['DATE'])

In [168]:
def join_by_date(df: pd.DataFrame, weather_df: pd.DataFrame, airport_name: str):
    tmp = df.loc[df['NAME'] == airport_name]
    tmp = tmp.drop(['NAME'], axis=1)
    weather_tmp = weather_df.loc[weather_df['NAME'] == airport_name]
    return pd.merge(tmp, weather_tmp, how='outer', on='DATE')

## Skip next frame if not merging with weather data

In [169]:
cleaned_df = pd.DataFrame()
weather_df = pd.read_csv('/mydata/wss-project-files/flight-delay-model/data/clean/clean_weather.csv')
weather_df['DATE'] = pd.to_datetime(weather_df['DATE'])
cities = ['JFK', 'DFW', 'MCO', 'LAX', 'SEA']
for city in cities:
    cleaned_df = cleaned_df.append(join_by_date(df, weather_df, city))


/tmp/ipykernel_83369/1151508563.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cleaned_df = cleaned_df.append(join_by_date(df, weather_df, city))
/tmp/ipykernel_83369/1151508563.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cleaned_df = cleaned_df.append(join_by_date(df, weather_df, city))
/tmp/ipykernel_83369/1151508563.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cleaned_df = cleaned_df.append(join_by_date(df, weather_df, city))
/tmp/ipykernel_83369/1151508563.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cleaned_df = cleaned_df.append(join_by_date(df, weather_df, city))
/tmp/ipykernel_83369/1151508563.py:6: Fu

## Skip next frame if you did not skip the previous

In [156]:
cleaned_df = df

In [170]:
cleaned_df = one_hot_encode(cleaned_df, 'NAME')

In [171]:
cleaned_df['DepDelay'] = [0 if temp <= 0 else 1 for temp in cleaned_df['DepDelay']]
cleaned_df['label'] = cleaned_df['DepDelay']
cleaned_df = cleaned_df.drop('DepDelay', axis=1)

In [172]:
columns = ['Unnamed: 0.1', 'Unnamed: 0_x', 'Unnamed: 0_y', 'Unnamed: 0', 'DATE']
for col in columns:
    try:
        cleaned_df = cleaned_df.drop([col], axis=1)
    except:
        pass
cleaned_df = cleaned_df.dropna(how='any')
cleaned_df = cleaned_df.reset_index(drop=True)

In [173]:
cleaned_df.to_csv('/mydata/wss-project-files/flight-delay-model/data/clean/clean_merge_full.csv')